In [ ]:
#| hide
from profane.core import *

# profane

> Not sacred

A hacky substitute for [sacred][], logging where, when and what experiment was run. As a comparison:

* [sacred][]: logs everything to MongoDB, requires integrating it into your code and using it to manage your config variables
* **profane**: logs everything to a local directory, requires no integration into your code, and doesn't care about your config variables beyond logging them

It's a wrapper for [weights & biases][wandb] so I don't have to write my own
terminal output capture solution. It just gets captured the same way it would be
if I was looking at it in the `wandb` dashboard. In addition, `wandb` captures
various metadata that I can just copy over, along with traces logging system
metrics (which `profane` has tools to extract to `.csv`).

This also means that the interface is the same as using `wandb`:

```
from profane.core import init

# somewhere in your script before you start your experiment:
run = init(
    project='my_project'
)
```

All kwargs are passed to `wandb` but `wandb` is run offline by default, set
`mode='online'` to change this.  This runs `wandb.init` internally so any future
calls to `wandb` will work as expected.

The difference from `wandb` is that when the run finishes the logs are copied to two locations:

* `local`: a local directory where **everything** captured gets saved; this is a backup of the wandb directory
* `shared`: a shared directory (I use a local git repo) where only the metadata and terminal logs get saved

Information about every run is saved and the logs are saved, nothing is lost,
and if I need to get more detail I can go look at the `local` directory and find
detailed information, such as all of the code if I enable [`settings=wandb.Settings(code_dir=".")`][wandbcode].

Another option to do something similar would be
[`script`](https://man7.org/linux/man-pages/man1/script.1.html) but it would
fail logging progress bars and other things that `wandb` captures without
problems. Also, it would require me to write my own solution for capturing
metadata.

[sacred]: https://github.com/IDSIA/sacred
[wandb]: https://github.com/wandb/wandb
[wandbcode]: https://docs.wandb.ai/guides/app/features/panels/code

## Install

I haven't put it on pypi yet so a dev install is recommended:

```sh
git clone git@github.com:gngdb/profane.git
cd profane
pip install -e .
```

## How to use

Before using `profane` run setup to set `local` and `shared` storage:

In [ ]:
!profane_setup --help

usage: Set up a ~/.profanerc file and register the required directories.
       [-h] [--user USER] local_storage shared_storage

positional arguments:
  local_storage   The local storage directory, everything will be stored here.
  shared_storage  The shared storage directory, only the terminal logs and
                  metadata will be stored here.

options:
  -h, --help      show this help message and exit
  --user USER     Optional: The user name to use for the shared storage
                  directory.


Then, as described above, `profane.core.init` can be used in place of [`wandb.init`](https://docs.wandb.ai/ref/python/init), all `kwargs` are passed to `wandb.init`. The only difference is the results are saved in the directories listed above.

In addition, profane kwargs may be provided prefixed by `profane_`. Currently, this includes:

* `profane_save` a list of paths to plaintext files that should be saved in the shared storage

```python
init(
    config: Union[Dict, str, None] = None,
    project: Optional[str] = None,
    entity: Optional[str] = None,
    tags: Optional[Sequence] = None,
    group: Optional[str] = None,
    name: Optional[str] = None,
    notes: Optional[str] = None,
    profane_save: Optional[Sequence] = None,
) -> Union[Run, RunDisabled, None]
```

`project` is used to name the directory in `shared` and `local` that will store whatever is logged.


## Example Logged

Example directory containing the metadata and logs from a run (the `shared` dir), logged with `profane`:

In [ ]:
from pathlib import Path

log_path = Path('./../offline-run-20230908_161004-d9b952ad')
print("- " + "\n- ".join(p.name for p in log_path.glob('*')))

- requirements.txt
- git.status
- config.json
- output.log
- wandb-summary.json
- argv.txt
- conda-environment.yaml
- system_stats.csv
- wandb-metadata.json
- git.patch


In [ ]:
from IPython.display import Markdown, display

for file in log_path.iterdir():
    if file.is_file():
        with open(file, 'r') as f:
            contents = f.read()
        display(Markdown(f"### {file.name}"))
        display(Markdown(f"```python\n{contents}\n```"))


### requirements.txt

```python
absl-py==1.4.0
appdirs==1.4.4
appnope==0.1.3
asttokens==2.2.1
attrs==23.1.0
backcall==0.2.0
backports.functools-lru-cache==1.6.4
beautifulsoup4==4.12.2
bleach==6.0.0
brotlipy==0.7.0
cachetools==5.3.1
certifi==2023.5.7
cffi==1.15.1
charset-normalizer==2.0.4
click==8.1.3
contourpy==1.0.5
cryptography==39.0.1
cycler==0.11.0
dahuffman==0.4.1
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.6
docker-pycreds==0.4.0
einops==0.6.1
executing==1.2.0
fastjsonschema==2.18.0
filelock==3.9.0
fonttools==4.25.0
fsspec==2023.6.0
gitdb==4.0.10
gitpython==3.1.31
gmpy2==2.1.2
google-auth-oauthlib==1.0.0
google-auth==2.21.0
grpcio==1.56.0
hjson==3.1.0
idna==3.4
importlib-metadata==6.6.0
ipykernel==6.15.0
ipython==8.14.0
jedi==0.18.2
jinja2==3.1.2
jsonschema-specifications==2023.7.1
jsonschema==4.19.0
jupyter-client==8.2.0
jupyter-core==5.3.0
jupyterlab-pygments==0.2.2
kiwisolver==1.4.4
markdown==3.4.3
markupsafe==2.1.3
matplotlib-inline==0.1.6
matplotlib==3.7.1
mistune==3.0.1
mpmath==1.2.1
munkres==1.1.4
mypy-extensions==1.0.0
nbclient==0.8.0
nbconvert==7.7.4
nbformat==5.9.2
nest-asyncio==1.5.6
networkx==2.8.4
ninja==1.11.1
numpy==1.24.3
oauthlib==3.2.2
opt-einsum==3.3.0
packaging==23.1
pandas==2.0.3
pandocfilters==1.5.0
parso==0.8.3
pathtools==0.1.2
pexpect==4.8.0
pickleshare==0.7.5
pillow==9.4.0
pip==23.0.1
platformdirs==2.5.2
profane==0.0.1
prompt-toolkit==3.0.38
protobuf==4.22.3
psutil==5.9.0
ptyprocess==0.7.0
pure-eval==0.2.2
py-cpuinfo==9.0.0
pyasn1-modules==0.3.0
pyasn1==0.5.0
pycparser==2.21
pydantic==1.10.9
pygments==2.15.1
pyopenssl==23.0.0
pyparsing==3.0.9
pyre-extensions==0.0.30
pysocks==1.7.1
python-dateutil==2.8.2
pytz==2023.3
pyyaml==6.0
pyzmq==25.1.0
referencing==0.30.2
requests-oauthlib==1.3.1
requests==2.29.0
rpds-py==0.9.2
rsa==4.9
seaborn==0.12.2
sentry-sdk==1.21.1
setproctitle==1.3.2
setuptools==66.0.0
six==1.16.0
smmap==5.0.0
soupsieve==2.4.1
stack-data==0.6.2
sympy==1.11.1
tensorboard-data-server==0.7.1
tensorboard==2.13.0
tinycss2==1.2.1
torch==2.1.0.dev20230428
torchaudio==2.1.0.dev20230428
torcheval==0.0.6
torchtnt==0.1.0
torchvision==0.16.0.dev20230428
tornado==6.2
tqdm==4.65.0
traitlets==5.9.0
typing-extensions==4.5.0
typing-inspect==0.9.0
tzdata==2023.3
urllib3==1.26.15
wandb==0.15.7
wcwidth==0.2.6
webencodings==0.5.1
werkzeug==2.3.6
wheel==0.38.4
x-transformers==1.16.9
zipp==3.15.0
```

### git.status

```python
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   main.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	wandb/

no changes added to commit (use "git add" and/or "git commit -a")

```

### config.json

...

```python
relative_seconds, gpu.0.gpu, gpu.0.memoryAllocated, gpu.0.temp, gpu.0.powerWatts, gpu.0.powerPercent, proc.memory.availableMB, memory, proc.memory.rssMB, proc.memory.percent, cpu, cpu.0.cpu_percent, cpu.1.cpu_percent, cpu.2.cpu_percent, cpu.3.cpu_percent, cpu.4.cpu_percent, cpu.5.cpu_percent, cpu.6.cpu_percent, cpu.7.cpu_percent, cpu.8.cpu_percent, cpu.9.cpu_percent, proc.cpu.threads, network.sent, network.recv, disk
0, 74.6, 31.54, 34.14, 6.37, 38.63, 3455.99, 78.9, 386.97, 2.36, 7.22, 29.36, 28.97, 55.53, 31.24, 13.1, 9.1, 5.21, 1.36, 0.71, 0.5, 15, 27940.57, 164717.71, 9.3
```

### wandb-metadata.json

```python
{
    "os": "macOS-13.5.1-arm64-arm-64bit",
    "python": "3.11.3",
    "heartbeatAt": "2023-09-08T20:10:04.564308",
    "startedAt": "2023-09-08T20:10:04.524040",
    "docker": null,
    "cuda": null,
    "args": [],
    "state": "running",
    "program": "/Users/gngdb/ws/pytorch_examples/mnist/main.py",
    "codePath": "mnist/main.py",
    "git": {
        "remote": "git@github.com:pytorch/examples.git",
        "commit": "cead596caa90600188e1055cd9166ab4e7dfd303"
    },
    "email": "***REMOVED***",
    "root": "/Users/gngdb/ws/pytorch_examples",
    "host": "Gavias-MacBook-Pro.local",
    "username": "gngdb",
    "executable": "/opt/homebrew/Caskroom/miniconda/base/envs/torch_nightly/bin/python",
    "cpu_count": 10,
    "cpu_count_logical": 10,
    "disk": {
        "total": 926.3517189025879,
        "used": 86.2264518737793
    },
    "gpuapple": {
        "type": "arm",
        "vendor": "Apple"
    },
    "memory": {
        "total": 16.0
    }
}

```

### git.patch

```python
diff --git a/mnist/main.py b/mnist/main.py
index 29d81d6..a95afa2 100644
--- a/mnist/main.py
+++ b/mnist/main.py
@@ -99,6 +99,10 @@ def main():
     use_cuda = not args.no_cuda and torch.cuda.is_available()
     use_mps = not args.no_mps and torch.backends.mps.is_available()
 
+    from profane.core import init
+
+    init(config=vars(args), project='mnist')
+
     torch.manual_seed(args.seed)
 
     if use_cuda:

```